In [1]:
import sys
sys.path.append('core')
import argparse
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from torch.utils.data import DataLoader
import datasets

In [3]:
def _pad_to_align(x, align):
    """Pad image batch x so width and height divide by align.

    Args:
      x: Image batch to align.
      align: Number to align to.

    Returns:
      1) An image padded so width % align == 0 and height % align == 0.
      2) A bounding box that can be fed readily to tf.image.crop_to_bounding_box
        to undo the padding.
    """
    # Input checking.
    assert np.ndim(x) == 4
    assert align > 0, "align must be a positive number."

    height, width = x.shape[-3:-1]
    height_to_pad = (align - height % align) if height % align != 0 else 0
    width_to_pad = (align - width % align) if width % align != 0 else 0

    bbox_to_pad = {
        "offset_height": height_to_pad // 2,
        "offset_width": width_to_pad // 2,
        "target_height": height + height_to_pad,
        "target_width": width + width_to_pad,
    }
    #  print(bbox_to_pad)
    padded_x = tf.image.pad_to_bounding_box(x, **bbox_to_pad)
    bbox_to_crop = {
        "offset_height": height_to_pad // 2,
        "offset_width": width_to_pad // 2,
        "target_height": height,
        "target_width": width,
    }
    return padded_x, bbox_to_crop


def flow_to_color(flow, mask=None, max_flow=None):
    """Converts flow to 3-channel color image.

    Args:
        flow: tensor of shape [num_batch, height, width, 2].
        mask: flow validity mask of shape [num_batch, height, width, 1].
        max_flow: unused argument.

    Returns:
      flow in 3-channel RGB image.

    Snippet Reference:
      Code: https://github.com/ppliuboy/SelFlow/blob/master/flowlib.py#L109
      MIT License: https://github.com/ppliuboy/SelFlow/blob/master/LICENSE
    """
    n = 8
    height, width, _ = tf.unstack(tf.shape(flow))
    mask = tf.ones([height, width, 1]) if mask is None else mask
    flow_u, flow_v = tf.unstack(flow, axis=2)
    if max_flow is not None:
        max_flow = tf.maximum(tf.to_float(max_flow), 1.0)
    else:
        max_flow = tf.reduce_max(tf.abs(flow * mask))
    mag = tf.sqrt(tf.reduce_sum(tf.square(flow), 2))
    angle = tf.atan2(flow_v, flow_u)

    im_h = tf.math.mod(angle / (2 * np.pi) + 1.0, 1.0)
    im_s = tf.clip_by_value(mag * n / max_flow, 0, 1)
    im_v = tf.clip_by_value(n - im_s, 0, 1)
    im_hsv = tf.stack([im_h, im_s, im_v], 2)
    im = tf.image.hsv_to_rgb(im_hsv)
    return im * mask


def flow_to_img(flow, normalize=True, info=None, flow_mag_max=None):
    """Convert flow to viewable image, using color hue to encode flow vector orientation, and color saturation to

    encode vector length. This is similar to the OpenCV tutorial on dense
    optical flow, except that they map vector
    length to the value plane of the HSV color model, instead of the saturation
    plane, as we do here.
    Args:
        flow: optical flow
        normalize: Normalize flow to 0..255
        info: Text to superimpose on image (typically, the epe for the predicted
          flow)
        flow_mag_max: Max flow to map to 255

    Returns:
        img: viewable representation of the dense optical flow in RGB format
        flow_avg: optionally, also return average flow magnitude
    Ref:
        - OpenCV 3.0.0-dev documentation » OpenCV-Python Tutorials » Video
        Analysis »
        https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html
        https://github.com/philferriere/tfoptflow/blob/master/tfoptflow/optflow.py#L190
        https://github.com/philferriere/tfoptflow/blob/master/LICENSE
    """
    hsv = np.zeros((flow.shape[0], flow.shape[1], 3), dtype=np.uint8)
    flow_magnitude, flow_angle = cv2.cartToPolar(
        flow[..., 0].astype(np.float32), flow[..., 1].astype(np.float32)
    )

    # A couple times, we've gotten NaNs out of the above...
    nans = np.isnan(flow_magnitude)
    if np.any(nans):
        nans = np.where(nans)
        flow_magnitude[nans] = 0.0

    # Normalize
    hsv[..., 0] = flow_angle * 180 / np.pi / 2
    if normalize is True:
        if flow_mag_max is None:
            hsv[..., 1] = cv2.normalize(flow_magnitude, None, 0, 255, cv2.NORM_MINMAX)
        else:
            hsv[..., 1] = flow_magnitude * 255 / flow_mag_max
    else:
        hsv[..., 1] = flow_magnitude
    hsv[..., 2] = 255
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    # Add text to the image, if requested
    if info is not None:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, info, (20, 20), font, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

    return img

In [4]:
def calculate_optical_flow_raft(loaded_model, image0, image1):
    image0 = tf.cast(image0, tf.float32) / 128.5 - 1
    image1 = tf.cast(image1, tf.float32) / 128.5 - 1

    image0_batch = np.reshape(image0, (1,) + image0.shape)
    image1_batch = np.reshape(image1, (1,) + image1.shape)
    image0_batch = tf.convert_to_tensor(image0_batch, dtype=tf.float32)
    image1_batch = tf.convert_to_tensor(image1_batch, dtype=tf.float32)

    image0_batch, bbox_to_crop = _pad_to_align(image0_batch, align=64)
    image1_batch, bbox_to_crop = _pad_to_align(image1_batch, align=64)

    input_batch = tf.concat([image0_batch, image1_batch], axis=0)
    input_batch = tf.expand_dims(input_batch, axis=0)

    flow_output = loaded_model(input_1=input_batch, input_2=tf.constant(12))

    flow_output = tf.image.crop_to_bounding_box(flow_output["output_1"], **bbox_to_crop)

    height, width, _ = image0.shape
    flow_shape = flow_output.shape
    np.testing.assert_array_equal(flow_shape, (1,) + (height, width, 2))

    return flow_output.numpy()[0]


def run_raft_it(model, im0_path, im1_path):
    im0 = media.read_image(im0_path)
    im1 = media.read_image(im1_path)
    optflow = calculate_optical_flow_raft(model.signatures["serving_default"], im0, im1)
    media.show_images([im0, im1, flow_to_img(optflow)])
    return optflow

In [ ]:
results = {}
for dstype in ['final']:
    val_dataset = datasets.MpiSintel(split='training', dstype=dstype)
    epe_list = []

    for val_id in range(len(val_dataset)):
        images, flow_gt, _ = val_dataset[val_id]
        image0, image1 = images[0], images[1]

        flow_low, flow_pr = model(images, iters=iters, test_mode=True)
        flow = padder.unpad(flow_pr[0]).cpu()

        epe = torch.sum((flow - flow_gt)**2, dim=0).sqrt()
        epe_list.append(epe.view(-1).numpy())

    epe_all = np.concatenate(epe_list)
    epe = np.mean(epe_all)
    px1 = np.mean(epe_all<1)
    px3 = np.mean(epe_all<3)
    px5 = np.mean(epe_all<5)

    print("Validation sintel (%s) EPE: %f, 1px: %f, 3px: %f, 5px: %f, >3px: %f" % (dstype, epe, px1, px3, px5, (1-px3)*100))
    results[dstype] = np.mean(epe_list)

In [5]:
import tensorflow as tf
model = tf.compat.v2.saved_model.load("raft_it_ft", options=tf.saved_model.LoadOptions(allow_partial_checkpoint=True))

NameError: name 'dstype' is not defined

In [ ]:
results = {}
for dstype in ['final']:
    val_dataset = datasets.MpiSintel(split='training', dstype=dstype)
    epe_list = []

    for val_id in range(len(val_dataset)):
        images, flow_gt, _ = val_dataset[val_id]
        image0, image1 = images[0], images[1]
        
        

        epe = torch.sum((flow - flow_gt)**2, dim=0).sqrt()
        epe_list.append(epe.view(-1).numpy())

    epe_all = np.concatenate(epe_list)
    epe = np.mean(epe_all)
    px1 = np.mean(epe_all<1)
    px3 = np.mean(epe_all<3)
    px5 = np.mean(epe_all<5)

    print("Validation sintel (%s) EPE: %f, 1px: %f, 3px: %f, 5px: %f, >3px: %f" % (dstype, epe, px1, px3, px5, (1-px3)*100))
    results[dstype] = np.mean(epe_list)

In [109]:
import mediapy as media
val_dataset = datasets.MpiSintel(split='training', dstype="final")

images, flow_gt, _ = val_dataset[0]
images = images.permute(0,2,3,1)
im0, im1 = images[0], images[1]
optflow = calculate_optical_flow_raft(model.signatures['serving_default'], im0, im1)
# media.show_images([np.uint8(im0), np.uint8(im1), flow_to_img(optflow), flow_to_img(np.array(flow_gt).transpose((2,0,1)))])
# media.show_images([np.uint8(im0), flow_to_img(optflow), flow_to_img(np.array(flow_gt).transpose((1,2,0)))])

In [110]:
flow = np.array(optflow).transpose((2,0,1))
flow_gt = np.array(flow_gt)


NameError: name 'flow' is not defined

In [123]:
epe = np.sqrt(np.sum((optflow*16 - flow_gt)**2, axis=0))

In [124]:
epe.mean()

1.5081654